# Assignment 1 - Code Example - Part B

This achieves an accuracy of 93.26% on test set.

In [1]:
# provide some basic operators like matrix multiplication
import numpy as np

## Some Useful Classes

In [2]:
# base class
class Module:
    @property
    def params(self): # trainable parameters
        return []

    def __call__(self, *args, **kwargs):
        return self.forward( *args, **kwargs)

# sequential module
class Sequential(Module, list):
    def __init__(self, *module_lst):
        super().__init__(module_lst)
    
    @property
    def params(self):
        return sum([m.params for m in self], []) # concat all params
    
    def forward(self, x):
        y = x
        for module in self:
            y = module(y)
        return y
    
    def backward(self, dy):
        dx = dy
        for module in self[::-1]:
            dx = module.backward(dx)
        return dx

## Softplus

This implements the [Softplus](https://pytorch.org/docs/stable/generated/torch.nn.Softplus.html) function.

$y = \frac{1}{\beta} \ln(1+e^{\beta x})$

$y' = \frac{1}{1+e^{-\beta x}}$

Default: $\beta=1$

$e^{\beta x}$ might be too large and unstable; so we use linear function to approximate it when $\beta x$ is above the threshold $20$.

In [3]:
class Softplus(Module):
    def __init__(self, beta=1.0, threshold=20.0):
        assert beta > 0 and threshold > 0
        self.beta = beta
        self.threshold = threshold

    def forward(self, x):
        self.beta_x = self.beta * x # save the input for backward use
        y = np.log(1 + np.exp(self.beta_x)) / self.beta
        y_relu = np.where(x > 0, x, 0)
        return np.where(x < self.threshold, y, y_relu)
    
    def backward(self, dy):
        grad = 1 / (1 + np.exp(-self.beta_x))
        grad_relu = np.where(self.beta_x > 0, 1, 0)
        return dy * np.where(self.beta_x < self.threshold, grad, grad_relu)

## LinearNoBias

This implements the [Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer but without the bias term.

$y = x W^T$

$dy/dx = W$

$dy/dW = x$

In [4]:
class LinearNoBias(Module):
    def __init__(self, in_features, out_features):
        self.weight = (np.random.rand(out_features, in_features) * 2 - 1) / in_features ** 0.5
        self.weight_grad = np.zeros_like(self.weight)

    @property
    def params(self):
        return [dict(val=self.weight, grad=self.weight_grad)]

    def forward(self, x):
        self.x = x
        return x @ self.weight.T

    def backward(self, dy):
        self.weight_grad[:] = dy.T @ self.x
        return dy @ self.weight
    

## CrossEntropyLoss

This implements the [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) loss.


In [5]:
def onehot(x, num_classes=10):
    y = np.zeros([len(x), num_classes])
    y[np.arange(len(x)), x] = 1
    return y


class CrossEntropyLoss(Module):    
    def forward(self, x_logit, x_target):
        self.x_logit = x_logit
        self.x_target = x_target
        
        # softmax
        x_logit_sub = np.exp(x_logit - np.max(x_logit, axis=1, keepdims=True))
        x_softmax = x_logit_sub / np.sum(x_logit_sub, axis=1, keepdims=True)
        x_softmax = np.clip(x_softmax, min=1e-15) # to avoid zero values
        self.x_softmax = x_softmax # save for backward

        # loss of each item
        loss_x = -np.log(x_softmax)[np.arange(len(x_target)), x_target]

        # average
        return loss_x.mean()

    def backward(self, dy):
        return dy * (self.x_softmax - onehot(self.x_target)) / len(self.x_logit)

## Optimizer

In [6]:
class Optim:
    def __init__(self, params, lr=0.01):
        self.params = params
        self.lr = lr

    def zero_grad(self):
        for idx in range(len(self.params)):
            self.params[idx]["grad"][:] = 0.0

In [7]:
class SGD(Optim):
    def step(self):
        for idx in range(len(self.params)):
            self.params[idx]["val"] -= self.lr * self.params[idx]["grad"]

## Your Network

In [8]:
net = Sequential(
    LinearNoBias(784, 512), Softplus(),
    LinearNoBias(512, 256), Softplus(),
    LinearNoBias(256, 128), Softplus(),
    LinearNoBias(128, 10),
)

loss_fn = CrossEntropyLoss()

## Training

In [9]:
# torch and torchvision provide some very handy utilities for dataset loading
from torch.utils.data import DataLoader
import torchvision.datasets as tv_datasets
import torchvision.transforms as tv_transforms

In [10]:
# some experimental setup
num_epochs = 32
batch_size = 128
num_workers = 2
print_every = 100

In [11]:
# prepare datasets
dataset, loader = {}, {}
for data_type in ("train", "test"):
    is_train = data_type=="train"
    dataset[data_type] = tv_datasets.MNIST(
        root="./data", train=is_train, download=True, 
        transform=tv_transforms.Compose([ # preprocessing pipeline for input images
            tv_transforms.ToTensor(),
            tv_transforms.Normalize((0.1307,), (0.3081,)),
    ]))
    loader[data_type] = DataLoader(
        dataset[data_type], batch_size=batch_size, shuffle=is_train, num_workers=num_workers,
    )


In [12]:
optimizer = SGD(net.params)

for epoch in range(num_epochs):

    running_loss = 0.0
    for i, (img, target) in enumerate(loader["train"]):
        img, target = img.numpy(), target.numpy()
        img = img.reshape(-1, 784)
        
        loss = loss_fn(net(img), target)

        net.backward(loss_fn.backward(loss))
        optimizer.step()
        optimizer.zero_grad()

        # print statistics
        running_loss += loss.item()
        if i % print_every == print_every - 1:
            print(f"[epoch={epoch + 1:3d}, iter={i + 1:5d}] loss: {running_loss / print_every:.3f}")
            running_loss = 0.0

print("Finished Training")

[epoch=  1, iter=  100] loss: 2.287
[epoch=  1, iter=  200] loss: 2.224
[epoch=  1, iter=  300] loss: 1.940
[epoch=  1, iter=  400] loss: 1.274
[epoch=  2, iter=  100] loss: 0.770
[epoch=  2, iter=  200] loss: 0.669
[epoch=  2, iter=  300] loss: 0.596
[epoch=  2, iter=  400] loss: 0.553
[epoch=  3, iter=  100] loss: 0.509
[epoch=  3, iter=  200] loss: 0.477
[epoch=  3, iter=  300] loss: 0.451
[epoch=  3, iter=  400] loss: 0.442
[epoch=  4, iter=  100] loss: 0.431
[epoch=  4, iter=  200] loss: 0.427
[epoch=  4, iter=  300] loss: 0.414
[epoch=  4, iter=  400] loss: 0.406
[epoch=  5, iter=  100] loss: 0.398
[epoch=  5, iter=  200] loss: 0.389
[epoch=  5, iter=  300] loss: 0.380
[epoch=  5, iter=  400] loss: 0.391
[epoch=  6, iter=  100] loss: 0.374
[epoch=  6, iter=  200] loss: 0.365
[epoch=  6, iter=  300] loss: 0.385
[epoch=  6, iter=  400] loss: 0.358
[epoch=  7, iter=  100] loss: 0.364
[epoch=  7, iter=  200] loss: 0.352
[epoch=  7, iter=  300] loss: 0.349
[epoch=  7, iter=  400] loss

## Evaluate

In [13]:
# for each test image
correct, total = 0, 0
for img, target in loader["test"]:
    img, target = img.numpy(), target.numpy()
    img = img.reshape(-1, 784)
    
    # make prediction
    pred = net(img)
    
    # accumulate
    total += len(target)
    correct += (np.argmax(pred, axis=1) == target).sum()

print(f"Accuracy of the network on the {total} test images: {100 * correct / total:.2f}%")

Accuracy of the network on the 10000 test images: 93.26%
